In [2]:
import os
import requests

os.environ['DB_HOST'] = '127.0.0.1'
os.environ['DISABLE_ELK'] = '1'
os.environ['DEBUG'] = '1'

from anubis.app import create_app
from anubis.models import db, User, InClass, Class_
from anubis.models import Assignment, AssignmentRepo, AssignmentTest
from anubis.models import Submission, SubmissionTestResult, SubmissionBuild
from anubis.utils.redis_queue import enqueue_webhook_rpc
import json


def get(path, token=None, params=None) -> dict:
    headers = {}
    if token is not None:
        headers['token'] = token
    if params is None:
        params = {}
    response = requests.get('http://localhost:5000' + path, headers=headers, params=params)
    print('path={} params={} data={}\n'.format(path, params, json.dumps(response.json(), indent=2)))
    return response.json()


app = create_app()

with app.app_context():
    # Create
    db.create_all()

    # Yeet
    SubmissionTestResult.query.delete()
    SubmissionBuild.query.delete()
    Submission.query.delete()
    AssignmentRepo.query.delete()
    AssignmentTest.query.delete()
    InClass.query.delete()
    Assignment.query.delete()
    Class_.query.delete()
    User.query.delete()
    db.session.commit()

    # Create
    u = User(netid='jmc1283', github_username='juanpunchman', name='John Cunniff', is_admin=True)
    c = Class_(name='Intro to OS', class_code='CS-UY 3224', section='A', professor='Gustavo')
    ic = InClass(owner=u, class_=c)
    a = Assignment(name='Assignment1: uniq', pipeline_image="registry.osiris.services/anubis/assignment/1", hidden=False, release_date='2020-08-22', due_date='2020-08-22', class_=c)
    at1 = AssignmentTest(name='Long file test', assignment=a)
    at2 = AssignmentTest(name='Short file test', assignment=a)
    r = AssignmentRepo(owner=u, assignment=a, repo_url='https://github.com/juan-punchman/xv6-public.git')
    s1 = Submission(commit='2bc7f8d636365402e2d6cc2556ce814c4fcd1489', state='Enqueued', owner=u, assignment=a, repo=r)
    s2 = Submission(commit='0001', state='Enqueued', owner=u, assignment=a, repo=r)

    # Commit
    db.session.add_all([u, c, ic, a, at1, at2, s1, s2, r])
    db.session.commit()

    # Init models
    s1.init_submission_models()
    s2.init_submission_models()
    
    enqueue_webhook_rpc(s1.id)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
import json
with app.app_context():
    print(json.dumps(Submission.query.filter_by(commit="2bc7f8d636365402e2d6cc2556ce814c4fcd1489").first().data, indent=2))